# Find your business audiences by their behavior directly

## Introduction 

Any time we want to run a new business, we have to find our audiences, our potential customers. But there is a question how to find people interested in our business or our product. One of the ways is to find them from their behavior in social networks. In this case we are going to find people may interest in our Online Jewelry Store.

## Data

As we knew Foursquare social network is about places and peoples. They tell stories about places visited and they are talking about it. In the other hand we have access to search venues, users and their interests. So we are going to find user that like jewelry stores in our territory. They are our very possible customers and their friends may be our potential audiences.

## Methodology

### Our search starts with few known parameters:
- __What is about our business or our product? How to find venues related to us?__
  *  One of the accurate ways to find related venues is to indicate it categories. By this __[link](https://developer.foursquare.com/docs/resources/categories)__ provided by Foursquare, we can easily found our categories    to search venues.
  *  As you can see we find out category for Jewelry Store (4bf58dd8d48988d111951735), it will be used later in search requests.
- __Where we are going to find peoples? a City, a brought or a neighborhood?__
  *  We chose New York city as our territory to find our audiences. By a geocoders library we will find the geographical location of our territory as a latitude and a longitude.

In [2]:
#!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

address    = 'New york city'
geolocator = Nominatim(user_agent="foursquare_agent")
location   = geolocator.geocode(address)
latitude   = location.latitude
longitude  = location.longitude
print(latitude, longitude)

40.7308619 -73.9871558


- __A logical range in meters to find people? How far we want to go to find them?__
  *  As described by Foursquare there is a __[search API](https://developer.foursquare.com/docs/api/venues/search)__ to find venues. There are some ways to search an area. One of them is search by a latitude and a longitude and a radius in a circle, it can be easily used but there is a limit of 50 venues in result set. The second way to search venues will help us to overcome this limitation, search by SW (south-west corner) and NE (north-east corner) it has limitation of 50 but we can easily divide our area to many squares and search them by this way. But we face a new problem: “who we can convert latitude and longitude to meaningful dividing squares?” this problem is considered in many __[navigation documents](https://www.movable-type.co.uk/scripts/latlong.html)__; each meter displacement is about 0.00001 in latitude  or longitude. So we are going to found shape our grid and search in made squares one by one.
  * we assumed radius as 10000 meters and squares of 500 * 500

In [3]:
# @hidden_cell 
CLIENT_ID     = '01KKSMDEM22BQBCHKT4IL4ZE5MUURWK0XV0IMVDHQ2F53DJU' # your Foursquare ID
CLIENT_SECRET = 'CFWN4MO3VCP2E1MCA00FPZI3GOHLIOBYX4PAII3PNVSJ345N' # your Foursquare Secret
VERSION = '20190301'

In [4]:
import pandas           as pd             # library for data analsysis
import requests                           # library to handle requests
import numpy            as np             # library to handle data in a vectorized manner
from pandas.io.json import json_normalize # tranforming json file into a pandas dataframe library
venues_dataframe = None
venues_dataframe = pd.DataFrame(columns =['id','name','location.lat','location.lng','SW_lat','SW_lon','NE_lat','NE_lon'])
category = '4bf58dd8d48988d111951735'
Coef   = 0.00001
radius = 10000
step   = 500
Base_Point_lat = latitude  - (Coef * radius / 2)
Base_Point_lon = longitude - (Coef * radius / 2)
for x in range(0,int(radius/step)):
    for y in range(0,int(radius/step)):
        Str_Point_lat = round(Base_Point_lat + float(x)*Coef*float(step)  ,5)
        Str_Point_lon = round(Base_Point_lon + float(y)*Coef*float(step)  ,5)
        End_Point_lat = round(Base_Point_lat + float(x+1)*Coef*float(step),5)
        End_Point_lon = round(Base_Point_lon + float(y+1)*Coef*float(step),5)
        url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&v={}&categoryId={}&sw={},{}&ne={},{}&intent=browse'.format(CLIENT_ID,CLIENT_SECRET,VERSION,category,Str_Point_lat,Str_Point_lon,End_Point_lat,End_Point_lon)
        results = requests.get(url).json()
        try :
            venues = results['response']['venues']
        except:
            print("Error0:", url)
        try:
            temp = json_normalize(venues)[['id','name','location.lat','location.lng']]
            temp['SW_lat'] = Str_Point_lat
            temp['SW_lon'] = Str_Point_lon
            temp['NE_lat'] = End_Point_lat
            temp['NE_lon'] = End_Point_lon
            venues_dataframe =venues_dataframe.append(temp)
        except:
            pass

In [5]:
venues_dataframe.shape

(1144, 8)

Because of the daily limitation of Foursquare API calls it is better to store harvested data into a database. I decided to use online IBM DB2 service as we learned in __[Databases and SQL for Data Science](https://www.coursera.org/learn/sql-data-science/home/welcome)__ course.

In [6]:
# @hidden_cell 
dsn_driver   = "{IBM DB2 ODBC DRIVER}"
dsn_database = "BLUDB"                                                  # e.g. "BLUDB"
dsn_hostname = "dashdb-txn-sbox-yp-lon02-01.services.eu-gb.bluemix.net" # e.g.: "dashdb-txn-sbox-yp-dal09-04.services.dal.bluemix.net"
dsn_port     = "50000"                                                  # e.g. "50000" 
dsn_protocol = "TCPIP"                                                  # i.e. "TCPIP"
dsn_uid      = "rpf77734"                                               # e.g. "abc12345"
dsn_pwd      = "lrjlqhjc6mc+qwdp"                                       # e.g. "7dBZ3wWt9XN6$o0J"

In [7]:
import ibm_db
import ibm_db_dbi
dsn = (
    "DRIVER={0};"
    "DATABASE={1};"
    "HOSTNAME={2};"
    "PORT={3};"
    "PROTOCOL={4};"
    "UID={5};"
    "PWD={6};").format(dsn_driver, dsn_database, dsn_hostname, dsn_port, dsn_protocol, dsn_uid, dsn_pwd)

try:
    conn = ibm_db.connect(dsn, "", "")
    print ("Connected to database: ", dsn_database, "as user: ", dsn_uid, "on host: ", dsn_hostname)

except:
    print ("Unable to connect: ", ibm_db.conn_errormsg() )

Connected to database:  BLUDB as user:  rpf77734 on host:  dashdb-txn-sbox-yp-lon02-01.services.eu-gb.bluemix.net


In [8]:
# delete all previous data in TBVENUES
SQL_Script = 'delete from TBVENUES where 1=1'
try :
    ibm_db.exec_immediate(conn, SQL_Script)
except:
    print(SQL_Script)
    print(ibm_db.stmt_errormsg())

In [9]:
for row in venues_dataframe.itertuples():
    SQL_Script = '                                             \
   INSERT  INTO TBVENUES                                       \
        (  ID ,NAME ,LON ,LAT ,SW_LON ,SW_LAT ,NE_LON ,NE_LAT )\
   VALUES                                                      \
        (\'{}\',\'{}\',{},{},{},{},{},{}                       );'.format(row.id,row.name.replace('\'','\'\''),round(row._3,5),round(row._4,5),row.SW_lat,row.SW_lon,row.NE_lat,row.NE_lon)
    try :
        insertStmt = ibm_db.exec_immediate(conn, SQL_Script)
    except :
        print(SQL_Script)
        print(ibm_db.stmt_errormsg())

Now it’s time to find peoples who liked jewelry stores. It can be obtained by likes API, in this API by a venue id we can get users who liked a venue, we store venue id and user id in our database.

In [10]:
dataframe_likes = None
dataframe_likes = pd.DataFrame(columns =['VENUE_ID','USER_ID'])
for venue_id in venues_dataframe['id']:
    likes_url = 'https://api.foursquare.com/v2/venues/{}/likes?client_id={}&client_secret={}&v={}'.format(venue_id,CLIENT_ID, CLIENT_SECRET, VERSION)
    try:
        results = requests.get(likes_url).json()
        try:
            likes = results['response']['likes']['items']
            try:
                TEMP = json_normalize(likes)[['id']].rename(columns={'id':'USER_ID'})
                try:    
                    TEMP['VENUE_ID'] = venue_id
                    try:
                        dataframe_likes = dataframe_likes.append(TEMP[['VENUE_ID','USER_ID']])
                    except:
                        print('ERROR4 :',likes_url )
                except:
                    print('ERROR3 :',likes_url )
            except:
                print('ERROR2 :',likes_url )
        except:
            pass
    except:
        print('ERROR0 :',likes_url)

In [11]:
dataframe_likes.shape

(853, 2)

Now we can store Likes into our Database

In [12]:
# delete all previous data in TBLIKES
SQL_Script = '    \
DELETE            \
  FROM  TBLIKES   \
 WHERE  1=1       '
try :
    ibm_db.exec_immediate(conn, SQL_Script)
except:
    print(SQL_Script)
    print(ibm_db.stmt_errormsg())

In [13]:
for row in dataframe_likes.itertuples():
    SQL_Script = '                                             \
   INSERT                                                      \
     INTO  TBLIKES                                             \
        (  VENUE_ID , USER_ID                                 )\
   VALUES                                                      \
        (\'{}\',\'{}\'                                         );'.format(row.VENUE_ID,row.USER_ID)
    try :
        insertStmt = ibm_db.exec_immediate(conn, SQL_Script)
    except :
        print(SQL_Script)
        print(ibm_db.stmt_errormsg())

To get details about each person we are going to use __[get details of a user API](https://developer.foursquare.com/docs/api/users/details)__ by a user id stored in previous step. We are going to obtain first name, last name, gender, home city and page URL of these people.

In [14]:
# remove duplicate user_id s form  dataframe_likes
dataframe_likes.reset_index(inplace = True)
dataframe_likes.drop_duplicates(subset = 'USER_ID',inplace = True)
dataframe_likes.drop(columns=['index'], axis = 1,inplace = True)

In [15]:
dataframe_likes.shape

(758, 2)

In [ ]:
user_data = pd.DataFrame(columns =['id','firstName','lastName','gender','homeCity','canonicalUrl'])
for user_id in dataframe_likes['USER_ID']:
    user_url = 'https://api.foursquare.com/v2/users/{}?client_id={}&client_secret={}&v={}'.format(user_id,CLIENT_ID, CLIENT_SECRET, VERSION)
    results = requests.get(user_url).json()
    try:
        user = results['response']['user']
        user_firstname    = None
        user_lastname     = None
        user_gender       = None
        user_homeCity     = None
        user_canonicalUrl = None
        try:
            user_firstname    = user['firstName']
        except:
            user_firstname    = ''
        try:
            user_lastname     = user['lastName']
        except:
            user_lastname     = ''
        try:
            user_gender       = user['gender']
        except:
            user_gender       = ''
        try:
            user_homeCity     = user['homeCity']
        except:
            user_homeCity     = ''
        try:
            user_canonicalUrl = user['canonicalUrl']
        except:
            canonicalUrl   = ''
        user_data = user_data.append({'id':user_id ,'firstName':user_firstname ,'lastName':user_lastname ,'gender':user_gender ,'homeCity':user_homeCity ,'canonicalUrl':user_canonicalUrl},ignore_index=True)
    except:
        print('error :' , user_url)
    

In [24]:
user_data.shape

(0, 6)

In [19]:
# delete all previous data in TBUSERS
SQL_Script = '    \
DELETE            \
  FROM  TBUSERS   \
 WHERE  1=1;      '
try :
    ibm_db.exec_immediate(conn, SQL_Script)
except:
    print(SQL_Script)
    print(ibm_db.stmt_errormsg())

In [20]:
#remove emoji and smilies from first name and last name
import re
emoji_pattern = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           "]+", flags=re.UNICODE)
for row in user_data.itertuples():
    SQL_Script = '                                  \
    INSERT                                           \
      INTO  TBUSERS                                  \
         (  ID ,firstName , lastName ,gender         \
         ,  homeCity      , canonicalUrl            )\
    VALUES                                           \
         (\'{}\',\'{}\',\'{}\',\'{}\',\'{}\',\'{}\');'.format(row.id,emoji_pattern.sub(r'', row.firstName).replace('\'','\'\''),emoji_pattern.sub(r'', row.lastName ).replace('\'','\'\''),row.gender,row.homeCity,row.canonicalUrl)
    try :
        insertStmt = ibm_db.exec_immediate(conn, SQL_Script)
    except :
        print(SQL_Script)
        print(ibm_db.stmt_errormsg())

In [21]:
ibm_db.close(conn)

True

In [23]:
user_data.head()

,id,firstName,lastName,gender,homeCity,canonicalUrl
